In [6]:
#standard imports. Note: You must pip install nasdaqdatalink 1st

import os
import pandas as pd
import nasdaqdatalink
import requests

In [7]:
# Linking my API key to .env in the same folder. The key is stored in the folder without any quotations around it 

nasdaqdatalink.read_key(filename=".env.txt")

In [8]:
# The actual API call using the SDK.
# Instructions can be found here https://data.nasdaq.com/databases/ZILLOW/usage/quickstart/python
# Replace 'quandl' w/ 'nasdaqdatalink

data = nasdaqdatalink.get_table('ZILLOW/DATA', indicator_id='ZSFH', region_id='99999')

In [9]:
data.head(10)

,indicator_id,region_id,date,value
None,,,,
0,ZSFH,99999,2022-06-30,588761.0
1,ZSFH,99999,2022-05-31,589700.0
2,ZSFH,99999,2022-04-30,580347.0
3,ZSFH,99999,2022-03-31,579785.0
4,ZSFH,99999,2022-02-28,555858.0
5,ZSFH,99999,2022-01-31,539084.0
6,ZSFH,99999,2021-12-31,523382.0
7,ZSFH,99999,2021-11-30,510814.0
8,ZSFH,99999,2021-10-31,500405.0
